In [ ]:
from SyMBac.external.DeLTA.model import unet_seg
from SyMBac.external.DeLTA.data import saveResult_seg, predictGenerator_seg, postprocess
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imshow
from scipy.ndimage.morphology import distance_transform_edt
from PIL import Image
from numpy import asarray
import os
import tensorflow as tf
import tifffile
from skimage.exposure import rescale_intensity
import skimage
from skimage.morphology import opening
from glob import glob
from tifffile import imread, imsave
from tqdm.notebook import tqdm
from skimage.transform import resize
import pandas as pd
from csbdeep.utils import Path, normalize
from stardist import fill_label_holes, random_label_cmap
from skimage.measure import regionprops_table, label
lbl_cmap = random_label_cmap()
from natsort import natsorted

In [ ]:
DeLTA_data =  "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/"
image_dirs = glob("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/real_data/*.tif")
training_set = DeLTA_data
target_size = (192,32)
input_size = target_size + (1,)
process_size = 4096
# Files:

models = natsorted(glob(DeLTA_data + '/models/*'))
model_file = DeLTA_data + "/models/saved-model-frac-0.0005.hdf5"
model_file

In [ ]:
cell_data = pd.DataFrame()
for t in range(1):
    
    
    
    #inputs_folder = DeLTA_data + 'mother_machine/evaluation/preprocessed/img/'
    #outputs_folder = DeLTA_data + 'mother_machine/evaluation/seg_output/'

    # Load up model:
    model = unet_seg(input_size = input_size)
    model.load_weights(model_file)
    
    data = imread(image_dirs[103])[:1200,:256]#[:,:256,5:37]
    X = []
    for x in range(len(data)):
        X.append(data[x])

    requires_resize = True
    resize_shape = (192,32) # (320,64) (512,128) (256,64)
    if requires_resize:
        X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    cell_data_temp = pd.DataFrame()
    all_labels = []
    all_imgs = []
    for x in tqdm(range(len(X))):
        img = normalize(X[x], 1,99.8, axis=(0,1))
        masks = model.predict(img.reshape(1,192,32,1)).reshape(192,32)
        all_labels.append(masks)
        all_imgs.append(img)

    cell_data_temp = pd.DataFrame()
    properties = ["area","centroid","major_axis_length","minor_axis_length","label","orientation"]
    for x in range(len(all_labels)):
        try:
            #border_cleared_nms = clear_border(nms_labels,buffer_size=5).astype(bool)*nms_labels
            #all_labels_nms.append(border_cleared_nms)
            temp_data = pd.DataFrame(regionprops_table(label((all_labels[x]>0.96).astype(bool)*1),properties = properties))
            temp_data = temp_data.sort_values(by = "centroid-0").iloc[0]
            current_cell_data  = temp_data
            current_cell_data["time"] = int(x)
            cell_data_temp = cell_data_temp.append(current_cell_data)
        except:
            pass
    cell_data_temp.reset_index(drop=True);
    cell_data_temp["epoch"] = t
    plt.figure(figsize=(20,2))
    plt.plot((cell_data_temp["major_axis_length"].reset_index(drop=True)*0.06*2))
    plt.show()
    plt.close()
    cell_data = cell_data.append(cell_data_temp)

In [ ]:
import matplotlib,numpy
import pylab
# A random colormap for matplotlib
cmap = matplotlib.colors.ListedColormap ( numpy.random.rand ( 256,3))

In [ ]:
for x in range(len(all_imgs)):
    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(2,6))
    ax1.imshow(all_imgs[x],cmap="Greys_r")
    ax2.imshow(label(all_labels[x]>0.8),cmap,interpolation="nearest")
    ax1.axis("off")
    ax2.axis("off")
    plt.savefig("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/one_shot_diag/{}.jpeg".format(str(x).zfill(4)))
    plt.close()

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(label(all_labels[418]>0.99),cmap,interpolation="nearest")

## Segmentation of entire dataset

In [5]:
DeLTA_data =  "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/DeLTA_new_training_data_hist_fourier"
image_dirs = glob("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/real_data/*.tif")
training_set = DeLTA_data
target_size = (256,48)
input_size = target_size + (1,)
process_size = 4096
# Files:

models = natsorted(glob(DeLTA_data + '/models/*'))
epoch = 70
model_file = DeLTA_data + "/models/new_saved-model-{}.hdf5".format(str(epoch).zfill(2))
model_file
t_seg = 1200
model = unet_seg(input_size = input_size)
model.load_weights(model_file)
cell_data = pd.DataFrame()

In [6]:
try:
    savedir="../Segmented_Data_Full_New_Epoch_{}_Exponential_Model_hist_fourier".format(str(epoch).zfill(2))
    os.mkdir(savedir)
except:
    pass

In [7]:
for t in tqdm(range(len(image_dirs))):
    data = imread(image_dirs[t])[:t_seg,:256]#[:,:256,5:37]
    X = []
    for x in range(len(data)):
        X.append(data[x])
    requires_resize = True
    resize_shape = target_size # (320,64) (512,128) (256,64)
    if requires_resize:
        X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in (X)]

    n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
    axis_norm = (0,1)   # normalize channels independently
    # axis_norm = (0,1,2) # normalize channels jointly
    if n_channel > 1:
        print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    cell_data_temp = pd.DataFrame()
    all_imgs = []
    for x in (range(len(X))):
        img = normalize(X[x], 1,99.8, axis=(0,1))
        all_imgs.append(img)
    all_imgs = np.array(all_imgs)
    masks = model.predict(all_imgs.reshape((t_seg,)+target_size+(1,))).reshape((t_seg,)+target_size)
    output_data = np.zeros((2,)  +  masks.shape)
    for x in range(len(all_imgs)):
        output_data[1,x,:,:] = all_imgs[x]
    output_data[0,:,:,:] = masks
    tifffile.imsave("{}/trench_{}.tif".format(savedir,t),output_data)

  0%|          | 0/236 [00:00<?, ?it/s]

# Segmentation of variable data amount

In [ ]:
DeLTA_data =  "/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/delta_hist_mid_large/"
image_dirs = glob("/home/georgeos/Storage/Dropbox (Cambridge University)/Phase/real_data/*.tif")
training_set = DeLTA_data
target_size = (192,32)
input_size = target_size + (1,)
process_size = 4096
# Files:

models = natsorted(glob(DeLTA_data + '/models/*'))
model_files = glob(DeLTA_data+"/models/*frac*")
num_imgs = (np.array([0.005,0.001,0.0005,0.01,0.1,0.2,0.5,0.75,0.9])*2000).astype(int)

In [ ]:
for num in num_imgs:
    os.mkdir("../Variable_Data_Segmentations/{}".format(num))

In [ ]:
print(num_imgs)
(model_files)

In [ ]:
t_seg = 300
for nn in tqdm(range(nn,len(num_imgs))):
    model = unet_seg(input_size = input_size)
    model.load_weights(model_files[nn])
    cell_data = pd.DataFrame()

    for t in tqdm(range(len(image_dirs))):
    #inputs_folder = DeLTA_data + 'mother_machine/evaluation/preprocessed/img/'
    #outputs_folder = DeLTA_data + 'mother_machine/evaluation/seg_output/'

        # Load up model:


        data = imread(image_dirs[t])[:t_seg,:256]#[:,:256,5:37]
        X = []
        for x in range(len(data)):
            X.append(data[x])

        requires_resize = True
        resize_shape = (192,32) # (320,64) (512,128) (256,64)
        if requires_resize:
            X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in (X)]

        n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
        axis_norm = (0,1)   # normalize channels independently
        # axis_norm = (0,1,2) # normalize channels jointly
        if n_channel > 1:
            print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
        cell_data_temp = pd.DataFrame()
        all_imgs = []
        for x in (range(len(X))):
            img = normalize(X[x], 1,99.8, axis=(0,1))
            all_imgs.append(img)
        all_imgs = np.array(all_imgs)
        masks = model.predict(all_imgs.reshape(t_seg,192,32,1)).reshape(t_seg,192,32)
        output_data = np.zeros((2,)  +  masks.shape)
        for x in range(len(all_imgs)):
            output_data[1,x,:,:] = all_imgs[x]
        output_data[0,:,:,:] = masks
        tifffile.imsave("../Variable_Data_Segmentations/{}/trench_{}.tif".format(num_imgs[nn],t),output_data)